In this file, we first do the PCA and then check the precision and the recall.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing as scale
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from sklearn import mixture
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [2]:
input_file = [ "biflow_sparta.csv", "biflow_scan_sU.csv", 'biflow_scan_A.csv', "biflow_mqtt_bruteforce.csv"]
columns_to_drop_bi = ['proto', 'ip_src', 'ip_dst']

In [3]:
def pred(x, clfs, p_nodes, gms):
    p_x_node = np.zeros((len(x), len(clfs))) 
    p_y_given_node = np.zeros((len(x), len(clfs))) 
    for i in range(len(clfs)):
        p_x_node[:, i] = p_nodes[i] * np.exp(gms[i].score_samples(x))
        p_y_given_node[:, i] = clfs[i].predict_proba(x)[:, 1]
    
    p_y = p_y_given_node * p_x_node / (np.sum(p_x_node, axis=-1, keepdims=True) + 1e-10)

    p_y = np.hstack([1-np.sum(p_y, axis=-1, keepdims=True), p_y])

    return np.argmax(p_y, axis=-1)

SVM:

In [4]:
def Fed_SVC_learn(input_file, columns_to_drop_bi, random_state,  K, test_size = 0.25):
    
    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data.drop(columns = columns_to_drop_bi, inplace = True)
        data.drop(columns='is_attack')
        data_list.append(data)

    rs = random_state

    #we do PCA here

    total_data = pd.concat(data_list, ignore_index=True)
    pca = PCA(n_components=4)
    pca.fit(total_data)
    for i in range(len(input_file)):
        data_list[i] = pca.transform(data_list[i])

    clfs = [] # This is going to contain 14 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = pd.read_csv(input_file[i])
        y[i][data0.is_attack == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = test_size, random_state = rs)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = np.concatenate(x_train_list,axis=0)
    scaler = scale.StandardScaler().fit(x_train_full)

    for i in range(len(input_file)):
        x_train_list[i] = scaler.transform(x_train_list[i])

    for i in range(len(input_file)):
        classifier = SVC(kernel = 'rbf', random_state = 41, gamma='scale',max_iter=-1, probability=True)
        classifier.fit(x_train_list[i], y_train_list[i])
        clfs.append(classifier)

    total_n_samples = np.sum(n_samples)

    p_nodes =  np.array(n_samples) / total_n_samples


    for i in range(len(input_file)):
        x_test_list[i] = scaler.transform(x_test_list[i])

    x_test = np.vstack(x_test_list)
    y_test = np.hstack(y_test_list)
    x_train = np.vstack(x_train_list)
    y_train = np.hstack(y_train_list)

    from sklearn.mixture import GaussianMixture
    gms = []
    for i in range(len(input_file)):
        x = x_train_list[i]
        gm = GaussianMixture(n_components = K).fit(x)  
        gms.append(gm)

    prediction = pred(x_test, clfs, p_nodes, gms)

    correct = prediction == y_test
    accuracy = np.mean(correct)


    accs = []
    for i in range(len(input_file)+1):
        if len(correct[y_test==i]) == 0:
            accs.append(0)
        else:
            accs.append(np.mean(correct[y_test==i]))



    recalls = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            recalls.append(0)
        else:
            recalls.append(np.mean(correct[prediction==i]))

    return recalls, accs 


In [5]:
def test(epoch, input_file, columns_to_drop_bi, K):

   recall , precision = np.ones((epoch,len(input_file)+1))*0 , np.ones((epoch,len(input_file)+1))*0

   print('main process:')

   for rs in tqdm(range(epoch)):
      recalls, accs = Fed_SVC_learn(input_file = input_file, columns_to_drop_bi=columns_to_drop_bi, random_state=rs, K=K)
      recall[rs,:] = recalls
      precision[rs,:] = accs

   for i in range(len(input_file)+1):
      print('Precision mean=',np.mean(precision[:,i]))
      print('Precision std=',np.std(precision[:,i]))

   for i in range(len(input_file)+1):
      print('Recall mean of ',i,' =',np.mean(recall[:,i]))
      print('Recall std of ',i,' =',np.std(recall[:,i]))


In [6]:
test(epoch=20, input_file=input_file, columns_to_drop_bi=columns_to_drop_bi, K=15)

main process:



Precision mean= 0.99903329216703
Precision std= 0.0030676796325696167
Precision mean= 1.0
Precision std= 0.0
Precision mean= 0.9999195907445187
Precision std= 0.00010546132833212416
Precision mean= 0.9971699887458062
Precision std= 0.0006486508423184991
Precision mean= 0.9706960413000914
Precision std= 0.002058017383807487
Recall mean of  0  = 0.9952855269297813
Recall std of  0  = 0.0003815112715059807
Recall mean of  1  = 1.0
Recall std of  1  = 0.0
Recall mean of  2  = 0.9993389170648916
Recall std of  2  = 0.00023414303200288385
Recall mean of  3  = 0.9959878871320514
Recall std of  3  = 0.014956630369360343
Recall mean of  4  = 0.9999573257467993
Recall std of  4  = 0.0001860127571923506


kNN

In [8]:
def Fed_knn_learn(input_file, columns_to_drop_bi, random_state,  K, test_size = 0.25):
    
    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data.drop(columns = columns_to_drop_bi, inplace = True)
        data.drop(columns='is_attack')
        data_list.append(data)

    rs = random_state

    #we do PCA here

    total_data = pd.concat(data_list, ignore_index=True)
    pca = PCA(n_components=4)
    pca.fit(total_data)
    for i in range(len(input_file)):
        data_list[i] = pca.transform(data_list[i])

    clfs = [] # This is going to contain 14 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = pd.read_csv(input_file[i])
        y[i][data0.is_attack == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = test_size, random_state = rs)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = np.concatenate(x_train_list,axis=0)
    scaler = scale.StandardScaler().fit(x_train_full)

    for i in range(len(input_file)):
        x_train_list[i] = scaler.transform(x_train_list[i])

    for i in range(len(input_file)):
        classifier = KNeighborsClassifier()
        classifier.fit(x_train_list[i], y_train_list[i])
        clfs.append(classifier)

    total_n_samples = np.sum(n_samples)

    p_nodes =  np.array(n_samples) / total_n_samples


    for i in range(len(input_file)):
        x_test_list[i] = scaler.transform(x_test_list[i])

    x_test = np.vstack(x_test_list)
    y_test = np.hstack(y_test_list)
    x_train = np.vstack(x_train_list)
    y_train = np.hstack(y_train_list)

    from sklearn.mixture import GaussianMixture
    gms = []
    for i in range(len(input_file)):
        x = x_train_list[i]
        gm = GaussianMixture(n_components = K).fit(x)  
        gms.append(gm)

    prediction = pred(x_test, clfs, p_nodes, gms)

    correct = prediction == y_test
    accuracy = np.mean(correct)


    accs = []
    for i in range(len(input_file)+1):
        if len(correct[y_test==i]) == 0:
            accs.append(0)
        else:
            accs.append(np.mean(correct[y_test==i]))



    recalls = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            recalls.append(0)
        else:
            recalls.append(np.mean(correct[prediction==i]))

    return recalls, accs 

def test(epoch, input_file, columns_to_drop_bi, K):

   recall , precision = np.ones((epoch,len(input_file)+1))*0 , np.ones((epoch,len(input_file)+1))*0

   print('main process:')

   for rs in tqdm(range(epoch)):
      recalls, accs = Fed_knn_learn(input_file = input_file, columns_to_drop_bi=columns_to_drop_bi, random_state=rs, K=K)
      recall[rs,:] = recalls
      precision[rs,:] = accs

   for i in range(len(input_file)+1):
      print('Precision mean=',np.mean(precision[:,i]))
      print('Precision std=',np.std(precision[:,i]))

   for i in range(len(input_file)+1):
      print('Recall mean of ',i,' =',np.mean(recall[:,i]))
      print('Recall std of ',i,' =',np.std(recall[:,i]))

test(epoch=20, input_file=input_file, columns_to_drop_bi=columns_to_drop_bi, K=15)


main process:



Precision mean= 0.9997851119324409
Precision std= 9.692096438756425e-05
Precision mean= 1.0
Precision std= 0.0
Precision mean= 0.9999196673279016
Precision std= 0.00010538857390851933
Precision mean= 0.9999698950475733
Precision std= 7.166494843614127e-05
Precision mean= 0.9707098572713541
Precision std= 0.002057137975372695
Recall mean of  0  = 0.9958311047250927
Recall std of  0  = 0.0003114342175256897
Recall mean of  1  = 1.0
Recall std of  1  = 0.0
Recall mean of  2  = 0.9994554902436062
Recall std of  2  = 0.0003163879362673795
Recall mean of  3  = 0.9995183904566783
Recall std of  3  = 0.0003073134819531641
Recall mean of  4  = 0.9999857712009106
Recall std of  4  = 6.202189731844314e-05


In [10]:
def Fed_LR_learn(input_file, columns_to_drop_bi, random_state,  K, test_size = 0.25):
    
    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data.drop(columns = columns_to_drop_bi, inplace = True)
        data.drop(columns='is_attack')
        data_list.append(data)

    rs = random_state

    #we do PCA here

    total_data = pd.concat(data_list, ignore_index=True)
    pca = PCA(n_components=4)
    pca.fit(total_data)
    for i in range(len(input_file)):
        data_list[i] = pca.transform(data_list[i])

    clfs = [] # This is going to contain 14 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = pd.read_csv(input_file[i])
        y[i][data0.is_attack == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = test_size, random_state = rs)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = np.concatenate(x_train_list,axis=0)
    scaler = scale.StandardScaler().fit(x_train_full)

    for i in range(len(input_file)):
        x_train_list[i] = scaler.transform(x_train_list[i])

    for i in range(len(input_file)):
        classifier = LogisticRegression(max_iter=100)
        classifier.fit(x_train_list[i], y_train_list[i])
        clfs.append(classifier)

    total_n_samples = np.sum(n_samples)

    p_nodes =  np.array(n_samples) / total_n_samples


    for i in range(len(input_file)):
        x_test_list[i] = scaler.transform(x_test_list[i])

    x_test = np.vstack(x_test_list)
    y_test = np.hstack(y_test_list)
    x_train = np.vstack(x_train_list)
    y_train = np.hstack(y_train_list)

    from sklearn.mixture import GaussianMixture
    gms = []
    for i in range(len(input_file)):
        x = x_train_list[i]
        gm = GaussianMixture(n_components = K).fit(x)  
        gms.append(gm)

    prediction = pred(x_test, clfs, p_nodes, gms)

    correct = prediction == y_test
    accuracy = np.mean(correct)


    accs = []
    for i in range(len(input_file)+1):
        if len(correct[y_test==i]) == 0:
            accs.append(0)
        else:
            accs.append(np.mean(correct[y_test==i]))



    recalls = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            recalls.append(0)
        else:
            recalls.append(np.mean(correct[prediction==i]))

    return recalls, accs 

def test(epoch, input_file, columns_to_drop_bi, K):

   recall , precision = np.ones((epoch,len(input_file)+1))*0 , np.ones((epoch,len(input_file)+1))*0

   print('main process:')

   for rs in tqdm(range(epoch)):
      recalls, accs = Fed_LR_learn(input_file = input_file, columns_to_drop_bi=columns_to_drop_bi, random_state=rs, K=K)
      recall[rs,:] = recalls
      precision[rs,:] = accs

   for i in range(len(input_file)+1):
      print('Precision mean=',np.mean(precision[:,i]))
      print('Precision std=',np.std(precision[:,i]))

   for i in range(len(input_file)+1):
      print('Recall mean of ',i,' =',np.mean(recall[:,i]))
      print('Recall std of ',i,' =',np.std(recall[:,i]))

test(epoch=20, input_file=input_file, columns_to_drop_bi=columns_to_drop_bi, K=15)


main process:



Precision mean= 0.9876506085171499
Precision std= 0.003785463910724846
Precision mean= 1.0
Precision std= 0.0
Precision mean= 0.9987506024068207
Precision std= 0.0021311291362336015
Precision mean= 0.9972498269214393
Precision std= 0.0006324387229597188
Precision mean= 0.9706960413000914
Precision std= 0.002058017383807487
Recall mean of  0  = 0.9949905705822607
Recall std of  0  = 0.0006040269834236191
Recall mean of  1  = 1.0
Recall std of  1  = 0.0
Recall mean of  2  = 0.9967037314634022
Recall std of  2  = 0.0067733042221125245
Recall mean of  3  = 0.9438932142488138
Recall std of  3  = 0.01899538541178398
Recall mean of  4  = 1.0
Recall std of  4  = 0.0


In [ ]:
RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = random_state)

In [11]:
def Fed_RF_learn(input_file, columns_to_drop_bi, random_state,  K, test_size = 0.25):
    
    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data.drop(columns = columns_to_drop_bi, inplace = True)
        data.drop(columns='is_attack')
        data_list.append(data)

    rs = random_state

    #we do PCA here

    total_data = pd.concat(data_list, ignore_index=True)
    pca = PCA(n_components=4)
    pca.fit(total_data)
    for i in range(len(input_file)):
        data_list[i] = pca.transform(data_list[i])

    clfs = [] # This is going to contain 14 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = pd.read_csv(input_file[i])
        y[i][data0.is_attack == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = test_size, random_state = rs)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = np.concatenate(x_train_list,axis=0)
    scaler = scale.StandardScaler().fit(x_train_full)

    for i in range(len(input_file)):
        x_train_list[i] = scaler.transform(x_train_list[i])

    for i in range(len(input_file)):
        classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = random_state)
        classifier.fit(x_train_list[i], y_train_list[i])
        clfs.append(classifier)

    total_n_samples = np.sum(n_samples)

    p_nodes =  np.array(n_samples) / total_n_samples


    for i in range(len(input_file)):
        x_test_list[i] = scaler.transform(x_test_list[i])

    x_test = np.vstack(x_test_list)
    y_test = np.hstack(y_test_list)
    x_train = np.vstack(x_train_list)
    y_train = np.hstack(y_train_list)

    from sklearn.mixture import GaussianMixture
    gms = []
    for i in range(len(input_file)):
        x = x_train_list[i]
        gm = GaussianMixture(n_components = K).fit(x)  
        gms.append(gm)

    prediction = pred(x_test, clfs, p_nodes, gms)

    correct = prediction == y_test
    accuracy = np.mean(correct)


    accs = []
    for i in range(len(input_file)+1):
        if len(correct[y_test==i]) == 0:
            accs.append(0)
        else:
            accs.append(np.mean(correct[y_test==i]))



    recalls = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            recalls.append(0)
        else:
            recalls.append(np.mean(correct[prediction==i]))

    return recalls, accs 

def test(epoch, input_file, columns_to_drop_bi, K):

   recall , precision = np.ones((epoch,len(input_file)+1))*0 , np.ones((epoch,len(input_file)+1))*0

   print('main process:')

   for rs in tqdm(range(epoch)):
      recalls, accs = Fed_RF_learn(input_file = input_file, columns_to_drop_bi=columns_to_drop_bi, random_state=rs, K=K)
      recall[rs,:] = recalls
      precision[rs,:] = accs

   for i in range(len(input_file)+1):
      print('Precision mean=',np.mean(precision[:,i]))
      print('Precision std=',np.std(precision[:,i]))

   for i in range(len(input_file)+1):
      print('Recall mean of ',i,' =',np.mean(recall[:,i]))
      print('Recall std of ',i,' =',np.std(recall[:,i]))

test(epoch=20, input_file=input_file, columns_to_drop_bi=columns_to_drop_bi, K=15)


main process:



Precision mean= 0.9999882879407587
Precision std= 2.1739315276567074e-05
Precision mean= 1.0
Precision std= 0.0
Precision mean= 0.9999284812139357
Precision std= 0.00010436471842279804
Precision mean= 0.9999798892840884
Precision std= 6.033232701048584e-05
Precision mean= 0.9707235715550114
Precision std= 0.002063439536603464
Recall mean of  0  = 0.9958455362052823
Recall std of  0  = 0.0003127049646664392
Recall mean of  1  = 1.0
Recall std of  1  = 0.0
Recall mean of  2  = 0.999991111111111
Recall std of  2  = 3.8745768387040074e-05
Recall mean of  3  = 0.9999699879055823
Recall std of  3  = 7.144320011612166e-05
Recall mean of  4  = 0.9999149971911665
Recall std of  4  = 0.00015770601603145597


Finally, we test the Full data after PCA

LR Centrailized:

In [3]:
def Cen_LR_learn(input_file, columns_to_drop_bi, random_state,  K, test_size = 0.25):
    
    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data.drop(columns = columns_to_drop_bi, inplace = True)
        data.drop(columns='is_attack')
        data_list.append(data)

    rs = random_state

    #we do PCA here

    total_data = pd.concat(data_list, ignore_index=True)
    pca = PCA(n_components=4)
    pca.fit(total_data)
    for i in range(len(input_file)):
        data_list[i] = pca.transform(data_list[i])

    clfs = [] # This is going to contain 14 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = pd.read_csv(input_file[i])
        y[i][data0.is_attack == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = test_size, random_state = rs)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = np.concatenate(x_train_list,axis=0)
    scaler = scale.StandardScaler().fit(x_train_full)

    for i in range(len(input_file)):
        x_train_list[i] = scaler.transform(x_train_list[i])



    total_n_samples = np.sum(n_samples)

    p_nodes =  np.array(n_samples) / total_n_samples


    for i in range(len(input_file)):
        x_test_list[i] = scaler.transform(x_test_list[i])

    x_test = np.vstack(x_test_list)
    y_test = np.hstack(y_test_list)
    x_train = np.vstack(x_train_list)
    y_train = np.hstack(y_train_list)


    classifier = LogisticRegression(max_iter=100)
    classifier.fit(x_train, y_train)

    prediction = classifier.predict(x_test)
    correct = prediction == y_test
    accuracy = np.mean(correct)


    accs = []
    for i in range(len(input_file)+1):
        if len(correct[y_test==i]) == 0:
            accs.append(0)
        else:
            accs.append(np.mean(correct[y_test==i]))



    recalls = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            recalls.append(0)
        else:
            recalls.append(np.mean(correct[prediction==i]))

    return recalls, accs 

def test(epoch, input_file, columns_to_drop_bi, K):

   recall , precision = np.ones((epoch,len(input_file)+1))*0 , np.ones((epoch,len(input_file)+1))*0

   print('main process:')

   for rs in tqdm(range(epoch)):
      recalls, accs = Cen_LR_learn(input_file = input_file, columns_to_drop_bi=columns_to_drop_bi, random_state=rs, K=K)
      recall[rs,:] = recalls
      precision[rs,:] = accs

   for i in range(len(input_file)+1):
      print('Precision mean=',np.mean(precision[:,i]))
      print('Precision std=',np.std(precision[:,i]))

   for i in range(len(input_file)+1):
      print('Recall mean of ',i,' =',np.mean(recall[:,i]))
      print('Recall std of ',i,' =',np.std(recall[:,i]))

test(epoch=20, input_file=input_file, columns_to_drop_bi=columns_to_drop_bi, K=15)


main process:


/Users/irohayachiyo/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/irohayachiyo/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo


Precision mean= 0.9943426263658429
Precision std= 0.0007108911523247878
Precision mean= 1.0
Precision std= 0.0
Precision mean= 0.7939042957159888
Precision std= 0.005879766072339707
Precision mean= 0.852208907330481
Precision std= 0.043490327784445414
Precision mean= 0.9705177235955199
Precision std= 0.0021283581189608203
Recall mean of  0  = 0.9275315864129258
Recall std of  0  = 0.007884172304582547
Recall mean of  1  = 1.0
Recall std of  1  = 0.0
Recall mean of  2  = 0.9971150517067612
Recall std of  2  = 0.0025217858766502376
Recall mean of  3  = 0.9684834407362739
Recall std of  3  = 0.002686844980200555
Recall mean of  4  = 0.9998865237586998
Recall std of  4  = 0.0002087984970708694


/Users/irohayachiyo/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


KNN:

In [4]:
def Cen_knn_learn(input_file, columns_to_drop_bi, random_state,  K, test_size = 0.25):
    
    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data.drop(columns = columns_to_drop_bi, inplace = True)
        data.drop(columns='is_attack')
        data_list.append(data)

    rs = random_state

    #we do PCA here

    total_data = pd.concat(data_list, ignore_index=True)
    pca = PCA(n_components=4)
    pca.fit(total_data)
    for i in range(len(input_file)):
        data_list[i] = pca.transform(data_list[i])

    clfs = [] # This is going to contain 14 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = pd.read_csv(input_file[i])
        y[i][data0.is_attack == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = test_size, random_state = rs)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = np.concatenate(x_train_list,axis=0)
    scaler = scale.StandardScaler().fit(x_train_full)

    for i in range(len(input_file)):
        x_train_list[i] = scaler.transform(x_train_list[i])



    total_n_samples = np.sum(n_samples)

    p_nodes =  np.array(n_samples) / total_n_samples


    for i in range(len(input_file)):
        x_test_list[i] = scaler.transform(x_test_list[i])

    x_test = np.vstack(x_test_list)
    y_test = np.hstack(y_test_list)
    x_train = np.vstack(x_train_list)
    y_train = np.hstack(y_train_list)


    classifier = KNeighborsClassifier()
    classifier.fit(x_train, y_train)

    prediction = classifier.predict(x_test)
    correct = prediction == y_test
    accuracy = np.mean(correct)


    accs = []
    for i in range(len(input_file)+1):
        if len(correct[y_test==i]) == 0:
            accs.append(0)
        else:
            accs.append(np.mean(correct[y_test==i]))



    recalls = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            recalls.append(0)
        else:
            recalls.append(np.mean(correct[prediction==i]))

    return recalls, accs 

def test(epoch, input_file, columns_to_drop_bi, K):

   recall , precision = np.ones((epoch,len(input_file)+1))*0 , np.ones((epoch,len(input_file)+1))*0

   print('main process:')

   for rs in tqdm(range(epoch)):
      recalls, accs = Cen_knn_learn(input_file = input_file, columns_to_drop_bi=columns_to_drop_bi, random_state=rs, K=K)
      recall[rs,:] = recalls
      precision[rs,:] = accs

   for i in range(len(input_file)+1):
      print('Precision mean=',np.mean(precision[:,i]))
      print('Precision std=',np.std(precision[:,i]))

   for i in range(len(input_file)+1):
      print('Recall mean of ',i,' =',np.mean(recall[:,i]))
      print('Recall std of ',i,' =',np.std(recall[:,i]))

test(epoch=20, input_file=input_file, columns_to_drop_bi=columns_to_drop_bi, K=15)


main process:



Precision mean= 0.9997091104715137
Precision std= 0.0002241402390449872
Precision mean= 1.0
Precision std= 0.0
Precision mean= 1.0
Precision std= 0.0
Precision mean= 1.0
Precision std= 0.0
Precision mean= 0.9709164271564521
Precision std= 0.0020706295436132374
Recall mean of  0  = 0.9958831158051403
Recall std of  0  = 0.00030121514378944305
Recall mean of  1  = 0.9998162351716072
Recall std of  1  = 0.0002867443403187562
Recall mean of  2  = 0.9997056419450144
Recall std of  2  = 0.00031621710982571237
Recall mean of  3  = 0.999749592470548
Recall std of  3  = 0.00025205363253831065
Recall mean of  4  = 0.9988977583771719
Recall std of  4  = 0.0012227471013892778


RF:

In [5]:
def Cen_RF_learn(input_file, columns_to_drop_bi, random_state,  K, test_size = 0.25):
    
    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data.drop(columns = columns_to_drop_bi, inplace = True)
        data.drop(columns='is_attack')
        data_list.append(data)

    rs = random_state

    #we do PCA here

    total_data = pd.concat(data_list, ignore_index=True)
    pca = PCA(n_components=4)
    pca.fit(total_data)
    for i in range(len(input_file)):
        data_list[i] = pca.transform(data_list[i])

    clfs = [] # This is going to contain 14 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = pd.read_csv(input_file[i])
        y[i][data0.is_attack == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = test_size, random_state = rs)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = np.concatenate(x_train_list,axis=0)
    scaler = scale.StandardScaler().fit(x_train_full)

    for i in range(len(input_file)):
        x_train_list[i] = scaler.transform(x_train_list[i])



    total_n_samples = np.sum(n_samples)

    p_nodes =  np.array(n_samples) / total_n_samples


    for i in range(len(input_file)):
        x_test_list[i] = scaler.transform(x_test_list[i])

    x_test = np.vstack(x_test_list)
    y_test = np.hstack(y_test_list)
    x_train = np.vstack(x_train_list)
    y_train = np.hstack(y_train_list)


    classifier = RandomForestClassifier(random_state = random_state)
    classifier.fit(x_train, y_train)

    prediction = classifier.predict(x_test)
    correct = prediction == y_test
    accuracy = np.mean(correct)


    accs = []
    for i in range(len(input_file)+1):
        if len(correct[y_test==i]) == 0:
            accs.append(0)
        else:
            accs.append(np.mean(correct[y_test==i]))



    recalls = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            recalls.append(0)
        else:
            recalls.append(np.mean(correct[prediction==i]))

    return recalls, accs 

def test(epoch, input_file, columns_to_drop_bi, K):

   recall , precision = np.ones((epoch,len(input_file)+1))*0 , np.ones((epoch,len(input_file)+1))*0

   print('main process:')

   for rs in tqdm(range(epoch)):
      recalls, accs = Cen_RF_learn(input_file = input_file, columns_to_drop_bi=columns_to_drop_bi, random_state=rs, K=K)
      recall[rs,:] = recalls
      precision[rs,:] = accs

   for i in range(len(input_file)+1):
      print('Precision mean=',np.mean(precision[:,i]))
      print('Precision std=',np.std(precision[:,i]))

   for i in range(len(input_file)+1):
      print('Recall mean of ',i,' =',np.mean(recall[:,i]))
      print('Recall std of ',i,' =',np.std(recall[:,i]))

test(epoch=20, input_file=input_file, columns_to_drop_bi=columns_to_drop_bi, K=15)


main process:



Precision mean= 0.9975841412803736
Precision std= 0.0003330946007789417
Precision mean= 1.0
Precision std= 0.0
Precision mean= 1.0
Precision std= 0.0
Precision mean= 1.0
Precision std= 0.0
Precision mean= 0.980167749391337
Precision std= 0.001653922354897871
Recall mean of  0  = 0.9971847429895095
Recall std of  0  = 0.00024396573132938096
Recall mean of  1  = 1.0
Recall std of  1  = 0.0
Recall mean of  2  = 1.0
Recall std of  2  = 0.0
Recall mean of  3  = 0.9999899819675416
Recall std of  3  = 4.36675910993865e-05
Recall mean of  4  = 0.9829476074846781
Recall std of  4  = 0.0023514263774262576


In [3]:
def Cen_SVC_learn(input_file, columns_to_drop_bi, random_state,  K, test_size = 0.25):
    
    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data.drop(columns = columns_to_drop_bi, inplace = True)
        data.drop(columns='is_attack')
        data_list.append(data)

    rs = random_state

    #we do PCA here

    total_data = pd.concat(data_list, ignore_index=True)
    pca = PCA(n_components=4)
    pca.fit(total_data)
    for i in range(len(input_file)):
        data_list[i] = pca.transform(data_list[i])

    clfs = [] # This is going to contain 14 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = pd.read_csv(input_file[i])
        y[i][data0.is_attack == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = test_size, random_state = rs)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = np.concatenate(x_train_list,axis=0)
    scaler = scale.StandardScaler().fit(x_train_full)

    for i in range(len(input_file)):
        x_train_list[i] = scaler.transform(x_train_list[i])



    total_n_samples = np.sum(n_samples)

    p_nodes =  np.array(n_samples) / total_n_samples


    for i in range(len(input_file)):
        x_test_list[i] = scaler.transform(x_test_list[i])

    x_test = np.vstack(x_test_list)
    y_test = np.hstack(y_test_list)
    x_train = np.vstack(x_train_list)
    y_train = np.hstack(y_train_list)


    classifier = SVC(kernel = 'rbf', random_state = random_state + 1, gamma='scale',max_iter=-1, probability=True)
    classifier.fit(x_train, y_train)

    prediction = classifier.predict(x_test)
    correct = prediction == y_test
    accuracy = np.mean(correct)


    accs = []
    for i in range(len(input_file)+1):
        if len(correct[y_test==i]) == 0:
            accs.append(0)
        else:
            accs.append(np.mean(correct[y_test==i]))



    recalls = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            recalls.append(0)
        else:
            recalls.append(np.mean(correct[prediction==i]))

    return recalls, accs 

def test(epoch, input_file, columns_to_drop_bi, K):

   recall , precision = np.ones((epoch,len(input_file)+1))*0 , np.ones((epoch,len(input_file)+1))*0

   print('main process:')

   for rs in tqdm(range(epoch)):
      recalls, accs = Cen_SVC_learn(input_file = input_file, columns_to_drop_bi=columns_to_drop_bi, random_state=rs, K=K)
      recall[rs,:] = recalls
      precision[rs,:] = accs

   for i in range(len(input_file)+1):
      print('Precision mean=',np.mean(precision[:,i]))
      print('Precision std=',np.std(precision[:,i]))

   for i in range(len(input_file)+1):
      print('Recall mean of ',i,' =',np.mean(recall[:,i]))
      print('Recall std of ',i,' =',np.std(recall[:,i]))

test(epoch=20, input_file=input_file, columns_to_drop_bi=columns_to_drop_bi, K=15)


main process:
